In [21]:
import xarray as xr
import geopandas as gpd
import os
import rasterio
from rasterio.mask import mask
from rasterio.io import MemoryFile
from rasterio.transform import from_bounds
from shapely.geometry import box, mapping
import numpy as np
import shapely.geometry as sg

In [23]:
# Path to the input shapefile
input_shapefile = r'D:\Nile project\Nile\Nile_basin\01 Final basins and Sub-basins\Sub_Basins.shp'

# Output directory to save individual shapefiles
output_directory = r'C:\Users\gaurav.fouzdar\path_to_output_folder_1'

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Read the input shapefile
gdf = gpd.read_file(input_shapefile)

# Ensure the column "Sb_name" exists in the GeoDataFrame
if "Sb_name" not in gdf.columns:
    raise ValueError("Column 'Sb_name' not found in the input shapefile.")

# Iterate over each feature in the GeoDataFrame
for idx, row in gdf.iterrows():
    # Extract the value from the "Sb_name" column
    sb_name = row["Sb_name"]
    
    # Create a GeoDataFrame for the single feature
    single_feature_gdf = gpd.GeoDataFrame([row], columns=gdf.columns)
    
    # Define the output file path using the "Sb_name" value
    output_shapefile = os.path.join(output_directory, f'{sb_name}.shp')
    
    # Save the single feature as a new shapefile
    single_feature_gdf.to_file(output_shapefile)
    
    print(f'Saved feature {sb_name} to {output_shapefile}')

Saved feature BAS_Agawi to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_Agawi.shp
Saved feature BAS_Akobo to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_Akobo.shp
Saved feature BAS_downstream K Nynding to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_downstream K Nynding.shp
Saved feature BAS_K_Nanaam_Pibor to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_K_Nanaam_Pibor.shp
Saved feature BAS_LowerBaro downstream machar to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_LowerBaro downstream machar.shp
Saved feature BG_Gel to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BG_Gel.shp
Saved feature BG_Bahr el Ghazal_North to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BG_Bahr el Ghazal_North.shp
Saved feature BAS_Pibor_downstream to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BAS_Pibor_downstream.shp
Saved feature BG_Bahr el Arab to C:\Users\gaurav.fouzdar\path_to_output_folder_1\BG_Bahr el Arab.shp
Saved feature BAS_LowerBaro adura to C:\Users\gaura

In [19]:
# Load the shapefile
shapefile_path = r'C:\Users\gaurav.fouzdar\path_to_output_folder\feature_33.shp'
gdf = gpd.read_file(shapefile_path)

# Check and set the CRS for the shapefile if it's not defined
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming the shapefile uses EPSG:4326 (lat/long)

# Load the NetCDF file with chunking
netcdf_path = r'C:\Users\gaurav.fouzdar\combined_rainfall_new.nc'
ds = xr.open_dataset(netcdf_path, chunks={'time': 100})

# Manually set the CRS for the NetCDF dataset
ds = ds.rio.write_crs("EPSG:4326")  # Replace with the correct CRS if known

# Ensure the CRS of the rainfall data and the shapefile are the same
if gdf.crs != ds.rio.crs:
    gdf = gdf.to_crs(ds.rio.crs)

# Add offset to the geometries for clipping
offset = 0.05  # Adjust this value as needed
geometries_offset = gdf.geometry.apply(lambda geom: geom.buffer(offset))

# Clip the rainfall data using the offset geometries
clipped_rainfall = ds.rio.clip(geometries_offset, ds.rio.crs)

# Save the clipped data to a new NetCDF file
clipped_netcdf_path = r'C:\Users\gaurav.fouzdar\clipped_rainfall_trial_1.nc'
clipped_rainfall.to_netcdf(clipped_netcdf_path)

print("Clipping with offset and saving complete.")

Clipping with offset and saving complete.


In [22]:
# Load the shapefile
shapefile_path = r'C:\Users\gaurav.fouzdar\path_to_output_folder\feature_33.shp'
gdf = gpd.read_file(shapefile_path)

# Check and set the CRS for the shapefile if it's not defined
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming the shapefile uses EPSG:4326 (lat/long)

# Load the NetCDF file with chunking
netcdf_path = r'C:\Users\gaurav.fouzdar\combined_rainfall_new.nc'
ds = xr.open_dataset(netcdf_path, chunks={'time': 100})

# Manually set the CRS for the NetCDF dataset
ds = ds.rio.write_crs("EPSG:4326")  # Replace with the correct CRS if known

# Ensure the CRS of the rainfall data and the shapefile are the same
if gdf.crs != ds.rio.crs:
    gdf = gdf.to_crs(ds.rio.crs)

# Add offset to the bounding box of the shapefile
offset = 0.05  # Adjust this value as needed
minx, miny, maxx, maxy = gdf.total_bounds
minx -= offset
miny -= offset
maxx += offset
maxy += offset

# Create a rectangular geometry with the offset extent
offset_extent = sg.box(minx, miny, maxx, maxy)

# Clip the rainfall data using the offset extent
clipped_rainfall = ds.rio.clip([offset_extent], ds.rio.crs)

# Save the clipped data to a new NetCDF file
clipped_netcdf_path = r'C:\Users\gaurav.fouzdar\clipped_rainfall_trial_extent.nc'
clipped_rainfall.to_netcdf(clipped_netcdf_path)

print("Clipping with offset extent and saving complete.")


Clipping with offset extent and saving complete.


In [ ]:
# Input and output directories
input_folder = r'C:\Users\gaurav.fouzdar\path_to_output_folder_1'
output_folder = r'C:\Users\gaurav.fouzdar\New folder'

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load the NetCDF file with chunking
netcdf_path = r'C:\Users\gaurav.fouzdar\combined_rainfall_new.nc'
ds = xr.open_dataset(netcdf_path, chunks={'time': 100})
ds = ds.rio.write_crs("EPSG:4326")  # Set the CRS for the NetCDF dataset

# Iterate through each shapefile in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.shp'):
        shapefile_path = os.path.join(input_folder, filename)
        gdf = gpd.read_file(shapefile_path)
        
        if gdf.crs is None:
            gdf.set_crs(epsg=4326, inplace=True)  # Set CRS if not defined
        
        if gdf.crs != ds.rio.crs:
            gdf = gdf.to_crs(ds.rio.crs)  # Ensure CRS match
        
        # Add offset to the bounding box of the shapefile
        offset = 0.1  # Adjust as needed
        minx, miny, maxx, maxy = gdf.total_bounds
        minx -= offset
        miny -= offset
        maxx += offset
        maxy += offset

        # Create a rectangular geometry with the offset extent
        offset_extent = sg.box(minx, miny, maxx, maxy)

        # Clip the rainfall data using the offset extent
        clipped_rainfall = ds.rio.clip([offset_extent], ds.rio.crs)

        # Get the base name of the shapefile without the extension
        base_name = os.path.splitext(filename)[0]

        # Save the clipped data to a new NetCDF file with the same base name as the shapefile
        output_path = os.path.join(output_folder, f'{base_name}_clipped.nc')
        clipped_rainfall.to_netcdf(output_path)
        print(f"Clipping and saving for {filename} complete.")